## Speed test for functions `combine_bincounts_kernel_weights` and `bin_counts_and_averages`

In [1]:
import numpy as np
import random
import numba
import math

For the *Local Instrumental Variable* (LIV) method, which will be added to the `grmpy` package, only the case with a single global bandwidth parameter is relevant. <br>
Currently, users have the option to provide an array or list of `M` local bandwidths, where `M`
is the number of equally-spaced gridpoint, as well. I might remove this feature, as it is not needed for `grmpy`. As a result, the variable `bwdisc` would be redundant.

In [2]:
np.random.seed(1111)

In [3]:
a = 0.005
b = 0.995
degree = 2 
bwdisc = 1
M = 500
bw = 0.322
tau = 4

In [4]:
pp = degree + 1
ppp = 2 * degree + 1
delta = (b - a) / (M - 1)

### 1) Function `combine_bincounts_kernel_weights`

In [5]:
L = math.floor(tau * bw / delta)
hdisc = bw

# Index set
indic = np.ones(M)

In [6]:
dimfkap = 2 * L + 1
midpt = L + 1

In [7]:
xcnts = np.random.uniform(0, 10, (M,))
ycnts = np.random.uniform(0, 10, (M,))

In [8]:
fkap = np.append(np.linspace(0.00035, 1, L+1), np.linspace(1, 0.00035, L+1)[1:])

In [9]:
def combine_bincounts_kernel_weights(
    xcnts, ycnts, bwdisc, M, pp, ppp, dimfkap, fkap, L, midpt, indic, delta
):
    """
    This function combines the bin counts (xcnts) and bin averages (ycnts) with
    kernel weights via a series of direct convolutions. As a result, binned
    approximations to X'W X and X'W y, denoted by ss and tt, are computed.

    Recall that the local polynomial curve estimator beta_ and its derivatives are
    minimizers to a locally weighted least-squares problem. At each grid
    point g = 1,..., M in the grid, beta_ is computed as the solution to the
    linear matrix equation:

    X'W X * beta_ = X'W y,

    where W are kernel weights approximated by the Gaussian density function.
    X'W X and X'W y are approximated by ss and tt,
    which are the result of a direct convolution of bin counts (xcnts) and kernel
    weights, and bin averages (ycnts) and kernel weights, respectively.

    The terms "kernel" and "kernel function" are used interchangeably
    throughout.

    For more information see the documentation of the main function locpoly
    under KernReg.locpoly.

    Parameters
    ----------
    xcnts: np.ndarry
        1-D array of binned x-values ("bin counts") of length M.
    ycnts: np.ndarry
        1-D array of binned y-values ("bin averages") of length M.
    bwdisc: int
        Number of logarithmically equally-spaced bandwidths
        on which local bandwidths are discretized, to speed up computation.
        For the case where 'bandwidth' is a scalar, bwdisc equals 1.
    M: int
        Gridsize, i.e. number of equally-spaced grid points.
    pp: int
        Number of columns of output array tt, i.e the binned approximation to X'W y.
        Degree + 1 (for degree = 2, pp equals 3).
    ppp: int
        Number of columns of output array ss, i.e. the binned approximation to X'W X.
        2 * degree + 1 (for degree = 2, ppp equals 5).
    dimfkap: int
        Length of 1-D array fkap.
    fkap: np.ndarry
        1-D array of length dimfkap containing
        approximated weights for the Gaussian kernel
        (W in the notation above).
    L: int or np.ndarry of length bwdisc
        Parameter defining the number of times the kernel function
        has to be evaluated.
        For the case where 'bandwidth' is a scalar, L is an integer.
        Note that L < N, where N is the total number of observations.
    midpt: int or np.ndarray of length bwdisc.
        For the case where 'bandwidth' is a scalar, midpt is an integer.
        Midpoint of fkap.
    indic: np.ndarry
        1-D array of length M containing ones.
        These are used as index values.
    delta: float
        Bin width.

    Returns
    -------
    ss: np.ndarry
        Dimensions (M, ppp). Binned approximation to X'W X.
    tt: np.ndarry
        Dimensions (M, pp). Binned approximation to X'W y.
    """
    ss = np.zeros((M, ppp))
    tt = np.zeros((M, pp))

    # Distinguish two cases:
    # a) Bandwidth is a scalar
    if bwdisc == 1:
        for g in range(M):
            if xcnts[g] != 0:
                for j in range(max(0, g - L - 1), min(M, g + L)):

                    # Only consider values within the range of fkap.
                    if (g - j + midpt - 1) in range(dimfkap):
                        if indic[j] == 1:

                            fac = 1

                            ss[j, 0] += xcnts[g] * fkap[g - j + midpt - 1]
                            tt[j, 0] += ycnts[g] * fkap[g - j + midpt - 1]

                            for ii in range(1, ppp):
                                fac = fac * delta * (g - j)

                                ss[j, ii] += xcnts[g] * fkap[g - j + midpt - 1] * fac

                                if ii < pp:
                                    tt[j, ii] += (
                                        ycnts[g] * fkap[g - j + midpt - 1] * fac
                                    )

    # b) Bandwidth is a list or np.ndarray of length M
    else:
        for g in range(M):
            if xcnts[g] != 0:

                # Repeat this process bwdisc times.
                for i in range(bwdisc):
                    for j in range(max(0, g - L[i] - 1), min(M, g + L[i])):

                        # Only consider values within the range of fkap.
                        if (g - j + midpt[i] - 1) in range(dimfkap):
                            if indic[j] == 1:

                                fac = 1

                                ss[j, 0] += xcnts[g] * fkap[g - j + midpt[i] - 1]

                                tt[j, 0] += ycnts[g] * fkap[g - j + midpt[i] - 1]

                                for ii in range(1, ppp):
                                    fac = fac * delta * (g - j)

                                    ss[j, ii] += (
                                        xcnts[g] * fkap[g - j + midpt[i] - 1] * fac
                                    )

                                    if ii < pp:
                                        tt[j, ii] += (
                                            ycnts[g] * fkap[g - j + midpt[i] - 1] * fac
                                        )

    return ss, tt

In [10]:
%%timeit 
ss, tt = combine_bincounts_kernel_weights(
    xcnts, ycnts, bwdisc, M, pp, ppp, dimfkap, fkap, L, midpt, indic, delta
)

2.75 s ± 171 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) Function `bin_counts_and_averages`

In [11]:
N = 1500
x = np.linspace(0.05, 0.97, N)
y = np.linspace(30, -7, N)

In [12]:
def bin_counts_and_averages(x, y, M=401, a=None, delta=None, truncate=True):
    """
    This function generates bin counts and bin averages over an equally spaced
    grid via the linear binning strategy.
    In essence, bin counts are obtained by assigning the raw data to
    neighboring grid points. A bin count can be thought of as representing the
    amount of data in the neighborhood of its corresponding grid point.
    Counts on the y-axis display the respective bin averages.

    The linear binning strategy is based on the linear transformation
    lxi = ((x - a) / delta) + 1, calculated for each observation
    x_i, i = 1, ... n. Its integer part, denoted by li, indicates the two
    nearest bin centers to x_i. This calculation already does the trick
    for simple binning. For linear binning, however, we additonally compute the
    "fractional part" or "remainder", lxi - li, which gives the weights
    attached to the two nearest bin centers, namely (1 - rem) for the bin
    considered and rem for the next bin.

    If trunctate is True, end observations are truncated.
    Otherwise, weight from end observations is given to corresponding
    end grid points.

    Parameters
    ----------
    x: np.ndarray
        Array of the predictor variable(s). Shape (N, r).
        Missing values are not accepted. Must be sorted.
    y: np.ndarray
        Array of the response variable of length N.
        Missing values are not accepted. Must come presorted by x.
    M: int
        Size of the grid mesh over which x and y are to be evaluated.
    a: float
        Start point of the grid.
    delta: float
        Bin width.
    truncate: bool
        If True, then endpoints are truncated.

    Returns
    -------
    xcounts: np.ndarry
        1-D array of binned x-values ("bin counts") of length M.
    ycounts: np.ndarry
        1-D array of binned y-values ("bin averages") of length M.
    """
    # Turn r-dimensional array into 1-d array.
    if x.ndim > 1:
        x = x.ravel()

    # Set bin width if not given.
    if delta is None:
        a = min(x)
        b = max(x)
        delta = (b - a) / (M - 1)

    n = len(x)

    xcounts = np.zeros(M)
    ycounts = np.zeros(M)
    lxi = np.zeros(n)
    li = np.zeros(n)
    rem = np.zeros(n)

    for i in range(n):
        lxi[i] = ((x[i] - a) / delta) + 1

        # Find integer part of "lxi"
        li[i] = int(lxi[i])
        rem[i] = lxi[i] - li[i]

    for g in range(M):
        # np.where(li == g) returns a tuple with the first element
        # being an np.ndarry containing indices. These indices denote where
        # an entry in li is equal to the respective gridpoint g.
        if len(np.where(li == g)[0]) > 0:
            # In case more than one entry in li euqals g,
            # go through all of them one by one.
            for j in range(len(np.where(li == g)[0])):

                xcounts[g - 1] += 1 - rem[np.where(li == g)[0][j]]
                xcounts[g] += rem[np.where(li == g)[0][j]]

                # If the predictor variable x is multidimensional and hence
                # len(x) is larger than len(y), consider only values
                # in range of y.
                if (np.where(li == g)[0][j]) in range(len(y)):
                    ycounts[g - 1] += (1 - rem[np.where(li == g)[0][j]]) * y[
                        np.where(li == g)[0][j]
                    ]

                    ycounts[g] += (
                        rem[np.where(li == g)[0][j]] * y[np.where(li == g)[0][j]]
                    )

    # By default, end observations are truncated.
    for i in range(n):
        if truncate is True:
            pass

        # Truncation is implicit if there are no points in li
        # beyond the grid's boundary points.
        elif 1 <= li[i] < M:
            pass

        # If truncate=False, weight from end observations is given to
        # corresponding end grid points.
        elif li[i] < 1 and truncate is False:
            if len(np.where(li < 1)[0]) > 0:
                for j in range(len(np.where(li < 1)[0])):
                    xcounts[0] += 1
                    ycounts[0] += [np.where(li < 1)[0][j]]

        elif li[i] >= M and truncate is False:
            if len(np.where(li == M)[0]) > 0:
                for j in range(len(np.where(li == M)[0])):
                    xcounts[M - 1] += 1
                    ycounts[M - 1] += y[np.where(li == M)[0][j]]

    return xcounts, ycounts

In [13]:
%timeit xcounts, ycounts = bin_counts_and_averages(x, y, M, a, delta)

295 ms ± 7.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
